In [5]:
def outliers_modified_z_score(ys): #outliers using z-score method
    threshold = 3.5
    median_y = np.median(ys)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in ys])
    modified_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y
                         for y in ys]
    return np.where(np.abs(modified_z_scores) > threshold)


def outliers_iqr(ys): #outliers using iqr method
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))

In [6]:
for idx,group in enumerate(df1.groupby(['Neighborhood'])):
    df_outliers_1 = pd.DataFrame()
    df_outliers_2 = pd.DataFrame()
    df_outliers_3 = pd.DataFrame()
    df_outliers_4 = pd.DataFrame()
    
    name_l = group[0]
    df_aux = group[1].copy()
    
    df_aux = df_aux[~((df_aux.LotArea == 0) | (df_aux.SalePrice == 0))]
    df_aux['price_sqm'] = df_aux.apply(lambda x: x.SalePrice / x.LotArea, axis=1) #price per sqm
    
    # remove outliers of area
    
    outliers_z_ar = outliers_modified_z_score(df_aux.LotArea.tolist())
    outliers_iqr_er_ar = outliers_iqr(df_aux.LotArea.tolist())
    
    if len(outliers_z_ar[0]) > 0:
        df_outliers_1 = df_aux.iloc[outliers_z_ar[0]]
        df_outliers_1['outtype'] ="sqm"

    if len(outliers_iqr_er_ar[0]) > 0:
        df_outliers_2 = df_aux.iloc[outliers_iqr_er_ar[0]]
        df_outliers_2['outtype'] ="sqm"

    if not df_outliers_1.empty:
        df_aux = df_aux[~df_aux.Id.isin(df_outliers_1.Id)]
        df_outliers_1.set_index('Id', inplace=True, drop=True)
    if not df_outliers_2.empty:
        df_aux = df_aux[~df_aux.Id.isin(df_outliers_2.Id)]
        df_outliers_2.set_index('Id', inplace=True, drop=True)

     # remove outliers of psqm
    
    outliers_z_ps = outliers_modified_z_score(df_aux.price_sqm.tolist())
    outliers_iqr_er_ps = outliers_iqr(df_aux.price_sqm.tolist())
    
    if len(outliers_z_ps[0]) > 0:
        df_outliers_3 = df_aux.iloc[outliers_z_ps[0]]
        df_outliers_3['outtype'] ="price"
    
    if len(outliers_iqr_er_ps[0]) > 0:
        df_outliers_4 = df_aux.iloc[outliers_iqr_er_ps[0]]
        df_outliers_4['outtype'] ="price"

    if not df_outliers_3.empty:
        df_aux = df_aux[~df_aux.Id.isin(df_outliers_3.Id)]
        df_outliers_3.set_index('Id', inplace=True, drop=True)
    if not df_outliers_4.empty:
        df_aux = df_aux[~df_aux.Id.isin(df_outliers_4.Id)]
        df_outliers_4.set_index('Id', inplace=True, drop=True)

    sns.set(style="darkgrid", color_codes=True)

    d = df_aux[['Id','price_sqm','LotArea']].copy()

    d.set_index('Id', inplace=True, drop=True)
    model = sm.OLS(d.price_sqm, d.LotArea)
    #model = sm.RLM(d.price_sqm, d.area, M=sm.robust.norms.HuberT())
    d['resid'] = model.fit().resid
    
    
    sns.jointplot(y='price_sqm', x='LotArea', data=d, kind="reg", color="r")

    
    head = d.sort_values(by=['resid'], ascending=[False]).head(5)
    tail = d.sort_values(by=['resid'], ascending=[False]).tail(5)
    

    plt.title(name_l, fontsize=10)
    plt.show()
        
    if not df_outliers_1.empty:
        print(df_outliers_1[['LotArea','price_sqm','outtype']])
    if not df_outliers_2.empty:
        print(df_outliers_2[['LotArea','price_sqm','outtype']])
    if not df_outliers_3.empty:
        print(df_outliers_3[['LotArea','price_sqm','outtype']])
    if not df_outliers_4.empty:
        print(df_outliers_4[['LotArea','price_sqm','outtype']])
        
    print(pd.concat([head,tail]))
    
    
    print('SELECT * FROM estates e where e.id in ({})'.format(','.join(str(x) for x in head.index)))
    
    if not df_outliers_1.empty:
        tot=pd.DataFrame(df_outliers_1[['LotArea','price_sqm','outtype']])
    if not df_outliers_2.empty:
        if not df_outliers_1.empty:
            tot=tot.append(df_outliers_2[['LotArea','price_sqm','outtype']])
        else:
            tot=pd.DataFrame(df_outliers_2[['LotArea','price_sqm','outtype']])
    if not df_outliers_3.empty:
        if not df_outliers_2.empty:
            tot=tot.append(df_outliers_3[['LotArea','price_sqm','outtype']])
        else:
            tot=pd.DataFrame(df_outliers_3[['LotArea','price_sqm','outtype']])
    if not df_outliers_4.empty:
        if not df_outliers_3.empty:
            tot=tot.append(df_outliers_4[['LotArea','price_sqm','outtype']])
        else:
            tot=pd.DataFrame(df_outliers_4[['LotArea','price_sqm','outtype']])

NameError: name 'df1' is not defined